# Setup

## Dependencies

In [1]:
# ! uv pip install kneed scikit-learn scipy pyoso plotly numpy 

In [2]:
from dotenv import load_dotenv
import os

from pyoso import Client
import pandas as pd

from datetime import datetime
import numpy as np
from pandas.tseries.offsets import DateOffset
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import plotly.express as px

pd.set_option('future.no_silent_downcasting', True)

## Authenticate to pyoso

In [3]:
load_dotenv()
stringify = lambda arr: "'" + "','".join(arr) + "'"
client = Client(api_key=os.environ['OSO_API_KEY'])
client.to_pandas("SHOW CATALOGS")

'SHOW CATALOGS' contains unsupported syntax. Falling back to parsing as a 'Command'.


,Catalog
0,iceberg
1,oso__gsheets


## Global settings

In [4]:
START_DATE = '2022-01-01'
END_DATE   = datetime.now().strftime('%Y-%m-%d')

BLUE_CHIP_TVL_THRESHOLD      = 100_000_000   # US$
MIN_LIFETIME_CONTRACT_INVOCS = 1_000
MASTER_MODEL_MONTHS          = 36
KMEANS_CLUSTERS              = 3

CHAINS = [
    'ARENAZ','AUTOMATA','BASE','BOB','CYBER','FRAX','HAM','INK','KROMA',
    'LISK','LYRA','METAL','MINT','MODE','OPTIMISM','ORDERLY','POLYNOMIAL',
    'RACE','REDSTONE','SHAPE','SONEIUM','SWAN','SWELL','UNICHAIN',
    'WORLDCHAIN','XTERIO','ZORA'
]
CHAIN_NAME_MAP = {'OPTIMISM': 'OPM'}
METRIC_DEFS = {
    'defillama_tvl_daily': {'label': 'TVL (US$ - Daily Avg)', 'aggfunc': 'mean'},
    'gas_fees_daily': {'label': 'Gas Fees (L2 ETH)', 'aggfunc': 'sum'},
    'transactions_daily': {'label': 'Transactions (Top-Level Only)', 'aggfunc': 'sum'},
    'contract_invocations_daily': {'label': 'Contract Invocations', 'aggfunc': 'sum'},
    'active_addresses_aggregation_daily': {'label': 'Active Addresses (Daily Avg)', 'aggfunc': 'mean'},
}
METRIC_KEYS=[
    'defillama_tvl_daily','gas_fees_daily','transactions_daily',
    'contract_invocations_daily','active_addresses_aggregation_daily'
]
METRIC_NAMES= [f"{c}_{m}" for c in CHAINS for m in METRIC_KEYS]
LABEL_MAP={m:METRIC_DEFS[m]["label"] for m in METRIC_DEFS}

In [5]:
ACTIVE_CATEGORIES = ['Blue-Chips', 'Established', 'High-Activity', 'Emerging-Projects', 'Low-Activity']

CLUSTER_NAMES = {
    'blue_chip': 'Blue-Chips',
    'established': 'Established',
    'high_activity': 'High-Activity',
    'emerging': 'Emerging-Projects',
    'inactive': 'Low-Activity',
    'not_active': 'Not Active / New'
}

ORDERED_CLUSTER_NAMES = [
    CLUSTER_NAMES['blue_chip'],
    CLUSTER_NAMES['established'],
    CLUSTER_NAMES['high_activity'],
    CLUSTER_NAMES['emerging'],
    CLUSTER_NAMES['inactive'],
    CLUSTER_NAMES['not_active']
]

# Queries

## Get projects

In [6]:
df_projects_raw = client.to_pandas("""
SELECT DISTINCT
    p.project_id,
    p.project_name AS oso_slug,
    p.display_name
FROM projects_v1 AS p
JOIN int_artifacts_by_project_in_ossd AS a ON a.project_id=p.project_id
WHERE a.artifact_type IN ('DEPLOYER', 'CONTRACT', 'DEFILLAMA_PROTOCOL')
""")
    
PROJECT_IDS = list(df_projects_raw['project_id'].unique())
PROJECT_NAMES = df_projects_raw.set_index('project_id')['display_name'].to_dict()

## Get metrics

In [7]:
df_metrics_raw = client.to_pandas(f"""

SELECT
    tm.project_id,
    tm.sample_date,
    m.metric_name,
    tm.amount
FROM timeseries_metrics_by_project_v0 AS tm
JOIN metrics_v0 AS m ON m.metric_id=tm.metric_id
WHERE
    tm.project_id IN ({stringify(PROJECT_IDS)})
    AND tm.sample_date BETWEEN DATE '{START_DATE}' AND DATE '{END_DATE}'
    AND m.metric_name IN ({stringify(METRIC_NAMES)})
""")

df_metrics_raw['amount'] = pd.to_numeric(df_metrics_raw['amount'],errors='coerce')
df_metrics_raw.tail()

,project_id,sample_date,metric_name,amount
2268555,L1FGc2AIMWcbDulPkpgzndWnljElZEAxxr+jPOdn0WQ=,2022-01-20,OPTIMISM_defillama_tvl_daily,144642804.42202
2268556,ExxaaR22Tn/4Q9iPwgzHc0FARRe+To/T1HFPcSJqi68=,2022-01-20,OPTIMISM_defillama_tvl_daily,53749061.87047
2268557,VFqE84jb9IOqejH667409R+gcJMUhhVSasXes3QZ/ZQ=,2022-01-20,OPTIMISM_defillama_tvl_daily,1392449.19378
2268558,ziS9zVwL0wejwpRl3exXS/sPkVTxes5OeRkH1buQ4Y4=,2022-01-20,OPTIMISM_defillama_tvl_daily,8724924.58747
2268559,YHiJ6aGP9ewErBaAX0mIXKnffPEH5HMfEv+nFn0QuHU=,2022-01-20,OPTIMISM_defillama_tvl_daily,11063469.92125


## Get funding histories

In [8]:
df_funding=client.to_pandas("""
    SELECT
        p.project_id,
        p.project_name AS oso_slug,
        f.funding_date,
        CASE
          WHEN f.grant_pool_name LIKE '%retro%' THEN 'Retro Funding'
          ELSE 'Gov Grants'
        END AS funding_program,
        JSON_EXTRACT(f.metadata,'$.token_amount') AS amount,
        JSON_EXTRACT(f.metadata,'$.application_name') AS application_name,
        JSON_EXTRACT(f.metadata,'$.application_url') AS url
    FROM stg_ossd__current_funding AS f
    JOIN projects_v1 AS p ON f.to_project_name=p.project_name
    WHERE f.from_funder_name='optimism'
""")
df_funding['amount']=pd.to_numeric(df_funding['amount'],errors='coerce')
df_funding=df_funding[df_funding['amount']>0]
df_funding['application_name']=df_funding['application_name'].str.replace('"','')
df_funding['url']=df_funding['url'].str.replace('"','')
df_funding['funding_date']=pd.to_datetime(df_funding['funding_date'])
df_funding.tail()

,project_id,oso_slug,funding_date,funding_program,amount,application_name,url
1931,9IE7nBPNl0g93qBfjjCAq2KUNwJeGPrzEeWsLxbolDM=,merkletreejs,2025-05-02,Retro Funding,29620.02,MerkleTreeJS,https://atlas.optimism.io/project/op_atlas_id ...
1932,9IE7nBPNl0g93qBfjjCAq2KUNwJeGPrzEeWsLxbolDM=,merkletreejs,2025-04-05,Retro Funding,32182.15,MerkleTreeJS,https://atlas.optimism.io/project/op_atlas_id ...
1933,9IE7nBPNl0g93qBfjjCAq2KUNwJeGPrzEeWsLxbolDM=,merkletreejs,2023-04-01,Retro Funding,58260.0,MerkleTreeJS,https://app.optimism.io/retropgf-discovery/0x3...
1934,Q3otD2dVMuz3OUvC1yAKkIrl9KIt8ksR1rOXG/vLvfI=,orbiter-finance,2025-06-09,Retro Funding,242.15,Orbiter,https://atlas.optimism.io/project/op_atlas_id ...
1935,Q3otD2dVMuz3OUvC1yAKkIrl9KIt8ksR1rOXG/vLvfI=,orbiter-finance,2023-04-01,Retro Funding,6697.0,Orbiter Finance,https://app.optimism.io/retropgf-discovery/0xd...


# Transforms

## Filter projects

In [9]:
tvl_by_project = (
    df_metrics_raw[df_metrics_raw['metric_name'].str.contains('tvl')]
    .groupby('project_id')['amount']
    .max()
)
tvl_by_project = tvl_by_project[tvl_by_project>0]
invocations_by_project = (
    df_metrics_raw[df_metrics_raw['metric_name'].str.contains('contract_invocations')]
    .groupby('project_id')['amount']
    .sum()
)
invocations_by_project = invocations_by_project[invocations_by_project>MIN_LIFETIME_CONTRACT_INVOCS]

relevant_project_ids = set(tvl_by_project.index).union(set(invocations_by_project.index))

In [10]:
df_project_summary = (
    df_projects_raw[df_projects_raw['project_id'].isin(relevant_project_ids)]
    .set_index('project_id')
    .join(
        df_funding.groupby('project_id').agg(
            rewards_sum=('amount','sum'),
            num_grants=('funding_date','nunique'),
            first_grant_date=('funding_date','min'),
            most_recent_grant_date=('funding_date','max'),
            received_gov_grant=('funding_program',lambda x:(x=='Gov Grants').any()),
            received_retro_funding=('funding_program',lambda x:(x=='Retro Funding').any()),
        )
    )
)
df_project_summary['rewards_sum'] = df_project_summary['rewards_sum'].fillna(0)
df_project_summary['num_grants'] = df_project_summary['num_grants'].fillna(0)
df_project_summary['received_gov_grant'] = df_project_summary['received_gov_grant'].fillna(False)
df_project_summary['received_retro_funding'] = df_project_summary['received_retro_funding'].fillna(False)
df_project_summary.tail(1)

,oso_slug,display_name,rewards_sum,num_grants,first_grant_date,most_recent_grant_date,received_gov_grant,received_retro_funding
project_id,,,,,,,,
UP1CZduWIgDJysUtiSKzySeyLCf8ZoZD5PdWYPKiOIw=,fxhash,fxhash,76281.728295,2.0,2024-07-17,2025-04-05,False,True


## Filter metrics

In [11]:
df_metrics = df_metrics_raw[
    (df_metrics_raw['project_id'].isin(relevant_project_ids)) &
    (df_metrics_raw['sample_date'] >= START_DATE) &
    (df_metrics_raw['sample_date'] < END_DATE)
].copy()

def extract_source_metric(metric_name):
    for m in METRIC_KEYS:
        if metric_name.endswith(f"_{m}"):
            return metric_name[0:-(len(m)+1)], m
    return None, None

df_metrics['source'], df_metrics['metric'] = zip(*df_metrics['metric_name'].map(extract_source_metric))
df_metrics['metric_label'] = df_metrics['metric'].map(LABEL_MAP)

is_tvl = df_metrics['metric'] == 'defillama_tvl_daily'
df_metrics.loc[is_tvl, 'amount'] = df_metrics.loc[is_tvl, 'amount'].replace(0, np.nan)

df_metrics['sample_date'] = pd.to_datetime(df_metrics['sample_date'])
df_metrics['time_period'] = df_metrics['sample_date'].dt.to_period('M').dt.start_time

df_metrics.tail()

,project_id,sample_date,metric_name,amount,source,metric,metric_label,time_period
2268555,L1FGc2AIMWcbDulPkpgzndWnljElZEAxxr+jPOdn0WQ=,2022-01-20,OPTIMISM_defillama_tvl_daily,144642804.42202,OPTIMISM,defillama_tvl_daily,TVL (US$ - Daily Avg),2022-01-01
2268556,ExxaaR22Tn/4Q9iPwgzHc0FARRe+To/T1HFPcSJqi68=,2022-01-20,OPTIMISM_defillama_tvl_daily,53749061.87047,OPTIMISM,defillama_tvl_daily,TVL (US$ - Daily Avg),2022-01-01
2268557,VFqE84jb9IOqejH667409R+gcJMUhhVSasXes3QZ/ZQ=,2022-01-20,OPTIMISM_defillama_tvl_daily,1392449.19378,OPTIMISM,defillama_tvl_daily,TVL (US$ - Daily Avg),2022-01-01
2268558,ziS9zVwL0wejwpRl3exXS/sPkVTxes5OeRkH1buQ4Y4=,2022-01-20,OPTIMISM_defillama_tvl_daily,8724924.58747,OPTIMISM,defillama_tvl_daily,TVL (US$ - Daily Avg),2022-01-01
2268559,YHiJ6aGP9ewErBaAX0mIXKnffPEH5HMfEv+nFn0QuHU=,2022-01-20,OPTIMISM_defillama_tvl_daily,11063469.92125,OPTIMISM,defillama_tvl_daily,TVL (US$ - Daily Avg),2022-01-01


## Pivot and merge on project data

In [12]:
mean_metric_labels = [v['label'] for k, v in METRIC_DEFS.items() if v['aggfunc'] == 'mean']
sum_metric_labels = [v['label'] for k, v in METRIC_DEFS.items() if v['aggfunc'] == 'sum']

metrics_for_mean = df_metrics[df_metrics['metric_label'].isin(mean_metric_labels)]
metrics_for_sum = df_metrics[df_metrics['metric_label'].isin(sum_metric_labels)]

pivot_mean_ts = metrics_for_mean.pivot_table(index=['time_period', 'project_id'], columns='metric_label', values='amount', aggfunc='mean')
pivot_sum_ts = metrics_for_sum.pivot_table(index=['time_period', 'project_id'], columns='metric_label', values='amount', aggfunc='sum')

df_pivot_timeseries = pd.merge(pivot_mean_ts, pivot_sum_ts, on=['time_period', 'project_id'], how='outer')

all_labels_ordered = [v['label'] for v in METRIC_DEFS.values()]
df_pivot_timeseries = df_pivot_timeseries.reindex(columns=all_labels_ordered)

df_analysis_ready_timeseries = df_pivot_timeseries.join(
    df_project_summary,
    on='project_id',
    how='left'
).fillna(0).reset_index()

date_cols_to_fix=['first_grant_date','most_recent_grant_date','time_period']
for col in date_cols_to_fix:
    df_analysis_ready_timeseries[col]=pd.to_datetime(df_analysis_ready_timeseries[col],errors='coerce')
df_analysis_ready_timeseries.dropna(subset=['time_period'],inplace=True)
df_analysis_ready_timeseries.tail()

,time_period,project_id,TVL (US$ - Daily Avg),Gas Fees (L2 ETH),Transactions (Top-Level Only),Contract Invocations,Active Addresses (Daily Avg),oso_slug,display_name,rewards_sum,num_grants,first_grant_date,most_recent_grant_date,received_gov_grant,received_retro_funding
14166,2025-07-01,zBJPQUvEV1LPYcdGjNtqucZ21cm5J39PGsoY/OFueYw=,20167060.278241,0.0,0.0,0.0,0.0,tokemak,Tokemak,0.0,0.0,1970-01-01,1970-01-01,False,False
14167,2025-07-01,zJU4xCN0flU4CqdPrmacHMHz54G7RCtspWeXx9cbV5U=,1832569.796565,0.048019,5882.0,22867.0,25.389831,pool-together,PoolTogether,742041.474685,4.0,2022-11-01,2024-07-17,True,True
14168,2025-07-01,ziS9zVwL0wejwpRl3exXS/sPkVTxes5OeRkH1buQ4Y4=,2930611.403078,0.253351,16161.0,155619.0,10.15,perpetual-protocol,Perpetual Protocol,9159562.300399,2.0,2022-07-01,2024-07-17,True,True
14169,2025-07-01,zvs0eTiLolOMzVdxyQsuo5exWgC/xRNbgQO527wwpg4=,0.0,0.003136,187.0,250.0,3.8125,multicaller-vectorized,Multicaller (by Vectorized),100952.31963,5.0,2024-07-17,2025-06-09,False,True
14170,2025-07-01,zx+dJHqx4qo8wGWwXVPOG55B/omdoN9vrtUaK6+Tsy0=,0.0,0.012721,1560.0,5749.0,32.344828,pods-media,pods.media,67500.918825,5.0,2024-07-17,2025-06-09,False,True


# Clustering analysis

In [13]:
SNAPSHOT = df_analysis_ready_timeseries['time_period'].max()
MASTER_MONTHS = MASTER_MODEL_MONTHS
KMEANS_K = KMEANS_CLUSTERS
TVL_THRESH = BLUE_CHIP_TVL_THRESHOLD
FEATURE_KEYS = [
    'defillama_tvl_daily',
    'transactions_daily',
    'contract_invocations_daily',
    'active_addresses_aggregation_daily'
]
FEATURE_LABELS = [METRIC_DEFS[k]['label'] for k in FEATURE_KEYS]

In [14]:
def get_period_slice(df, start, end):
    mask = (df['time_period'] >= start) & (df['time_period'] < end)
    return df.loc[mask].copy()

def scale_features(df, labels, tvl_col='TVL (US$ - Daily Avg)'):
    # blue-chips & inactive
    df_copy = df.copy()
    blue     = df_copy[df_copy[tvl_col] >= TVL_THRESH].assign(cluster=CLUSTER_NAMES['blue_chip'])
    inactive = df_copy[df_copy['Contract Invocations'] < 100].assign(cluster=CLUSTER_NAMES['inactive'])
    to_cluster = df_copy[
        (df_copy[tvl_col] < TVL_THRESH) &
        (df_copy['Contract Invocations'] >= 100)
    ].copy()
    if to_cluster.empty:
        return None, blue, inactive, None, []

    # log + scale
    log_cols = []
    for lbl in labels:
        if lbl in to_cluster.columns:
            lc = f'log_{lbl}'
            to_cluster = to_cluster.copy()
            to_cluster.loc[:, lc] = np.log10(to_cluster[lbl] + 1e-6)
            log_cols.append(lc)
    if not log_cols:
        return None, blue, inactive, None, []

    X = StandardScaler().fit_transform(to_cluster[log_cols])
    return to_cluster, blue, inactive, X, log_cols


def train_master_model(df_master):
    df_c, blue, inactive, X, log_cols = scale_features(df_master, FEATURE_LABELS)
    kmeans = KMeans(n_clusters=KMEANS_K, random_state=42, n_init=10)
    kmeans.fit(X)
    centers = kmeans.cluster_centers_

    # map centroids → names
    ti  = log_cols.index('log_TVL (US$ - Daily Avg)')
    tx  = log_cols.index('log_Transactions (Top-Level Only)')
    est = np.argmax(centers[:, ti])
    rem = [i for i in range(KMEANS_K) if i != est]
    hub = rem[np.argmax(centers[rem, tx])]
    emerg = [i for i in rem if i != hub][0]

    cmap = {
        est: CLUSTER_NAMES['established'],
        hub: CLUSTER_NAMES['high_activity'],
        emerg: CLUSTER_NAMES['emerging']
    }
    return centers, cmap

def assign_clusters(df_period, centers, cmap):
    df_c, blue, inactive, X, _ = scale_features(df_period, FEATURE_LABELS)
    parts = []
    if df_c is not None:
        dists = distance.cdist(X, centers, 'euclidean')
        idx   = np.argmin(dists, axis=1)
        df_c  = df_c.copy()
        df_c.loc[:, 'cluster'] = [cmap[i] for i in idx]
        parts.append(df_c)
    parts += [blue, inactive]
    return pd.concat(parts, ignore_index=True, sort=False)

def build_cluster_history():
    history = []
    projects = df_analysis_ready_timeseries['project_id'].unique()
    periods = [
        (SNAPSHOT - DateOffset(months=i+1), SNAPSHOT - DateOffset(months=i))
        for i in range(MASTER_MONTHS)
    ]

    centers, cmap = train_master_model(
        get_period_slice(df_analysis_ready_timeseries,
                         SNAPSHOT - DateOffset(months=MASTER_MONTHS),
                         SNAPSHOT)
    )

    for start, end in reversed(periods):
        scaffold = pd.DataFrame({
            'project_id': projects,
            'snapshot_date': pd.to_datetime(end)
        })
        df_per = get_period_slice(df_analysis_ready_timeseries, start, end)
        if df_per is not None and not df_per.empty:
            assigned = assign_clusters(df_per, centers, cmap)
            assigned = assigned.copy()
            assigned.loc[:, 'snapshot_date'] = pd.to_datetime(end)
            merged = scaffold.merge(assigned,
                                    on=['project_id', 'snapshot_date'],
                                    how='left')
        else:
            merged = scaffold.assign(cluster=CLUSTER_NAMES['not_active'])

        merged = merged.assign(
            cluster=merged['cluster'].fillna(CLUSTER_NAMES['not_active'])
        )
        history.append(merged)

    return pd.concat(history, ignore_index=True)

In [15]:
df_builders_over_time = build_cluster_history()

primary_chain = (
    df_metrics
    .groupby('project_id')['source']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
    .rename('chain')
)
df_builders_over_time = (
    df_builders_over_time
    .merge(primary_chain, on='project_id', how='left')
    .assign(chain=lambda d: d['chain'].fillna('Unknown'))
)
df_builders_over_time.loc[:, 'snapshot_date'] = pd.to_datetime(df_builders_over_time['snapshot_date'])
df_builders_over_time.head()

,project_id,snapshot_date,time_period,TVL (US$ - Daily Avg),Gas Fees (L2 ETH),Transactions (Top-Level Only),Contract Invocations,Active Addresses (Daily Avg),oso_slug,display_name,...,first_grant_date,most_recent_grant_date,received_gov_grant,received_retro_funding,log_TVL (US$ - Daily Avg),log_Transactions (Top-Level Only),log_Contract Invocations,log_Active Addresses (Daily Avg),cluster,chain
0,1PqWi/6kQu3j9O311xKAqoA9vYRJjIlLefadHnswYrE=,2022-08-01,2022-07-01,0.0,0.040912,1007.0,55105.0,13.677419,fraxfinance,Frax Finance,...,2024-09-01,2024-09-01,True,False,-6.0,3.003029,4.741191,1.136004,High-Activity,OPTIMISM
1,1XmrMwdqgUb8jliAn9nIScPx7fAicOqIFPW42ioD1x0=,2022-08-01,2022-07-01,0.0,0.007906,12232.0,23209.0,292.483871,socket,Socket,...,2022-11-01,2025-06-09,True,True,-6.0,4.087497,4.365656,2.466102,High-Activity,OPTIMISM
2,2l+DcexWOerkhRsKXiaWZam/soE6IQ5tVe7t4K202lo=,2022-08-01,2022-07-01,201711.786529,0.116934,10834.0,88005.0,122.258065,thales,Thales,...,2022-07-01,2024-09-01,True,True,5.304731,4.034789,4.944507,2.087278,Established,OPTIMISM
3,6E3DyJA6elKGNzeUnrVYN/T9kutfmbzXXNbOlvSfnZM=,2022-08-01,2022-07-01,0.0,0.194276,6819.0,13990.0,198.83871,lifinance,LI.FI,...,2022-11-01,2025-06-09,True,True,-6.0,3.833721,4.145818,2.298501,High-Activity,OPTIMISM
4,6eLtF7Ky2IKk8PQYZW1+wxMpZsg9RqzFzDg3qMmYLnQ=,2022-08-01,2022-07-01,100135.56095,0.0017,240.0,1755.0,3.866667,kromatika-finance,Kromatika,...,2022-11-01,2024-01-01,True,True,5.000588,2.380211,3.244277,0.587337,Emerging-Projects,OPTIMISM


In [16]:
df_variance = (
    df_builders_over_time
    .groupby(['project_id', 'display_name'])['cluster']
    .nunique()
    .sort_values(ascending=False)
    .reset_index(name='cluster_changes')
)
print("Top 10 Unstable Builders:")
display(df_variance.head(10))

Top 10 Unstable Builders:


,project_id,display_name,cluster_changes
0,BvXTdDsg+jdxF9BynZy+IJejoLAolScBB2Sq78SsU20=,Router Protocol,4
1,SbFbagHrVLpnhXLPsZg6EYdenq1+y5k/aOkGXo32/Ww=,Bunni.xyz,4
2,PCuBi0AunZbM9RokjCH3on5O9Gz+ixkNQFpaxZA9zrY=,Spark Foundation,4
3,/48A4/LZsBf1ztk3blMoY9kPQjx8knNp/EabsEszQ4Y=,Wormhole Foundation,4
4,LpeVfrlQFe6kw3cYW8YO6+bSUZr6TJDU9YshbtC9OPM=,Hedgey Finance,4
5,JiV3uSqnMm/+EVWoAdKTOAKsY8UN4zpNahllPnRy8nk=,AlfaFrens,4
6,iIP3vV/VOUMzWW2bMRLS+GXybH89CIg0hhnzJjC7IOc=,Vesper,4
7,V9cLpIkU/z1qoFKorpO0P/HNgBI37ZexTmGZpHGUA1c=,Arcadia Finance,3
8,X9ntWd5Z5wZp/2ASYV7zaTY5DvqrazFcutm8vi+Yscc=,LottoPGF,3
9,WPkRYGeWT+KCYyfUaq4CK2HJV5FdO4G9T5lbvi/dA8o=,Vela Exchange,3
